# Working with Text Data
* Before we can implement and train LLMs in the upcoming chapters, we need to prepare the training dataset.
* This involves splitting text into individual word and subword tokens, which can then be encoded into vector representations for the LLM.
* Lastly, we'll implement a sampling and data loading strategy to produce the input-output pairs necessary for training LLMs in subsequent chapters.

## Understanding word embeddings


* Deep neural network models, including LLMs, cannot process raw text directly. Therefore, we need a way to represent words as continuous-valued vectors.
* The concept of converting data into a vector format is often referred to as embedding. Using a specific neural network layer or another pretrained neural network model, we can embed different data types, for example, video, audio, and text.
* Different data formats require distinct embedding models. For example, an embedding model designed for text would not be suitable for embedding audio or video data.
* This chapter focus on word embeddings. There are also embeddings for sentences, paragraphs, or whole documents. Sentence or paragraph embeddings are popular choices for retrieval-augmented generation
* One of the earlier and most popular examples to generate word embeddings is the Word2Vec approach.
  * Word2Vec trained neural network architecture to generate word embeddings by predicting the context of a word given the target word or vice versa.
  * The main idea behind Word2Vec is that words that appear in similar contexts tend to have similar meanings.
  * Consequently, when projected into 2-dimensional word embeddings for visualization purposes, it can be seen that similar terms cluster together.
* LLMs commonly produce their own embeddings that are part of the input layer and are updated during training. The advantage of optimizing the embeddings as part of the LLM training instead of using Word2Vec is that the embeddings are optimized to the specific task and data at hand.
* The smallest GPT-2 models (117M and 125M parameters) use an embedding size of 768 dimensions to provide concrete examples. The largest GPT-3 model (175B parameters) uses an embedding size of 12,288 dimensions.
*  Preparing the embeddings used by an LLM include splitting text into words, converting words into tokens, and turning tokens into embedding vectors.

## Tokenizing text

* This section covers how we split input text into individual tokens, a required preprocessing step for creating embeddings for an LLM. These tokens are either individual words or special characters, including punctuation characters.
* Next I follow the instruction of the book to work on text in the-verdict.txt. See chap2/token.ipynb.

## Converting tokens into token IDs

* Convert tokens from a Python string to an integer representation to produce the so-called token IDs is an intermediate step before converting the token IDs into embedding vectors.
* We have to build a so-called vocabulary first. This vocabulary defines how we map each unique word and special character to a unique integer.
* When we want to convert the outputs of an LLM from numbers back into text, we also need a way to turn token IDs into text. For this, we can create an inverse version of the vocabulary that maps token IDs back to corresponding text tokens.
* We need to consider large and diverse training sets to extend the vocabulary when working on LLMs.

## Adding special context tokens
* Modify the tokenizer to handle unknow words.
* Adding special tokens that can enhance a model's understanding of context or other relevant information in the text.
  * <|unk|> is used when we encounter words that is not part of the vocabulary.
  * <|endoftext|> is used between unrelated texts. For example, when training GPT-like LLMs on multiple independent documents or books, it is common to insert a token before each document or book that follows a previous text source.
* Depending on the LLM, some researchers also consider additional special tokens such as the following:
  * \[BOS\] (beginning of sequence): This token marks the start of a text. It signifies to the LLM where a piece of content begins.
  * \[EOS\] (end of sequence): This token is positioned at the end of a text, and is especially useful when concatenating multiple unrelated texts.
  * \[PAD\] (padding): When training LLMs with batch sizes larger than one, the batch might contain texts of varying lengths. To ensure all texts have the same length, the shorter texts are extended or "padded" using the \[PAD\] token, up to the length of the longest text in the batch.
* The tokenizer used for GPT models does not need any of these tokens mentioned above. Only use <|endoftext|> for \[EOS\] and padding.

## Byte pair encoding
* A more sophisticated tokenization scheme based on a concept called byte pair encoding (BPE).
* The BPE tokenizer covered in this section was used to train LLMs such as GPT-2, GPT-3, and the original model used in ChatGPT.
* The <|endoftext|> token is assigned a relatively large token ID, namely, 50256. total vocabulary size of 50,257, with <|endoftext|> being assigned the largest token ID.
* The BPE tokenizer above encodes and decodes unknown words, such as "someunknownPlace" correctly. The BPE tokenizer can handle any unknown word. How does it achieve this without using <|unk|> tokens?
* The algorithm underlying BPE breaks down words that aren't in its predefined vocabulary into smaller subword units or even individual characters, enabling it to handle out-of-vocabulary words.
* BPE builds its vocabulary by iteratively merging frequent characters into subwords and frequent subwords into words.
  * BPE starts with adding all individual single characters to its vocabulary ("a", "b", ...).
  * In the next stage, it merges character combinations that frequently occur together into subwords. For example, "d" and "e" may be merged into the subword "de," which is common in many English words like "define", "depend", "made", and "hidden".
  * The merges are determined by a frequency cutoff.

## Data sampling with a sliding window
* Generate the input-target pairs required for training an LLM.
* What do these input-target pairs look like? LLMs are pretrained by predicting the next word in a text.
* We implement a data loader that fetches the input-target pairs from the training dataset using a sliding window approach.
* One of the easiest and most intuitive ways to create the input-target pairs for the next-word prediction task is to create two variables, x and y, where x contains the input tokens and y contains the targets, which are the inputs shifted by 1.
* Implementing an efficient data loader that iterates over the input dataset and returns the inputs and targets as PyTorch tensors, which can be thought of as multidimensional arrays.
* In particular, we are interested in returning two tensors: an input tensor containing the text that the LLM sees and a target tensor that includes the targets for the LLM to predict.
* Define a class based on `Pytorch Dataset` class. Contain `__init__`, `__len__` and `__getitem__`.
* Load inputs in batches via Pytorch `DataLoader`.

## Creating token embeddings
* The last step for preparing the input text for LLM training is to convert the token IDs into embedding vectors.
* We initialize these embedding weights with random values as a preliminary step. This initialization serves as the starting point for the LLM's learning process.
* The weight matrix of the embedding layer contains small, random values. These values are optimized during LLM training as part of the LLM optimization itself, as we will see in upcoming chapters.
### Embedding layers versus matrix multiplication
* The embedding layer approach above is essentially just a more efficient way of implementing one-hot encoding followed by matrix multiplication in a fully connected layer.

## Encoding word positions
* A minor shortcoming of LLMs is that their self-attention mechanism, which will be covered in detail in chapter 3, doesn't have a notion of position or order for the tokens within a sequence.
* The way the previously introduced embedding layer works is that the same token ID always gets mapped to the same vector representation, regardless of where the token ID is positioned in the input sequence.
* It is helpful to inject additional position information into the LLM. There are two broad categories of position-aware embeddings: relative positional embeddings and absolute positional embeddings.
  * Absolute positional embeddings are directly associated with specific positions in a sequence.
  *  For each position in the input sequence, a unique embedding is added to the token's embedding to convey its exact location.
  * The emphasis of relative positional embeddings is on the relative position or distance between tokens. This means the model learns the relationships in terms of "how far apart" rather than "at which exact position."
* OpenAI's GPT models use absolute positional embeddings that are optimized during the training process.

## Summary
* As the first step, raw text is broken into tokens, which can be words or characters. Then, the tokens are converted into integer representations, termed token IDs.
* Special tokens, such as <|unk|> and <|endoftext|>, can be added to enhance the model's understanding and handle various contexts, such as unknown words or marking the boundary between unrelated texts.
* The byte pair encoding (BPE) tokenizer used for LLMs like GPT-2 and GPT-3 can efficiently handle unknown words by breaking them down into subword units or individual characters.
* We use a sliding window approach on tokenized data to generate input-target pairs for LLM training.
* Embedding layers in PyTorch function as a lookup operation, retrieving vectors corresponding to token IDs. The resulting embedding vectors provide continuous representations of tokens, which is crucial for training deep learning models like LLMs.
* While token embeddings provide consistent vector representations for each token, they lack a sense of the token's position in a sequence. To rectify this, two main types of positional embeddings exist: absolute and relative. OpenAI's GPT models utilize absolute positional embeddings that are added to the token embedding vectors and are optimized during the model training.